In [8]:
from elasticsearch import Elasticsearch
from elasticsearch_dsl import Document, InnerDoc, Nested, Text, Integer
from elasticsearch_dsl.connections import connections
import pandas as pd

es = Elasticsearch([{"host":"localhost","port":9200}])
connections.create_connection(hosts=['localhost'], timeout=20)

# Définition du Document Elasticsearch
class Video(Document):
    title = Text()
    channel = Text()
    views = Text()
    date = Text()

    class Index:
        name = 'video_index'

# Créer l'index dans Elasticsearch s'il n'existe pas
Video.init()

# Charger la dataframe Pandas
df = pd.DataFrame({0: ['Hogwarts Legacy - Official Launch Trailer 4K', 'Hogwarts Legacy Review', 'Hogwarts Legacy - Before You Buy'],
                1: ['Hogwarts Legacy', 'IGN', 'gameranx'],
                2: ['3,2 M de vues', '1,8 M de vues', '1,8 M de vues'],
                3: ['il y a 6 jours', 'il y a 2 jours', 'il y a 1 jour']})
df.columns = ['title', 'channel', 'views', 'date']

# Convertir la dataframe en objets Video
objs = [Video(title=row['title'], channel=row['channel'], views=row['views'], date=row['date'])
        for idx, row in df.iterrows()]

# Insérer les objets dans Elasticsearch
for obj in objs:
    obj.save()

results = Video.search().execute()
for result in results:
    print(result.title, result.channel, result.views, result.date)



Hogwarts Legacy - Official Launch Trailer 4K Hogwarts Legacy 3,2 M de vues il y a 6 jours
Hogwarts Legacy Review IGN 1,8 M de vues il y a 2 jours
Hogwarts Legacy - Before You Buy gameranx 1,8 M de vues il y a 1 jour


In [7]:
objs

[Video(index='video_index', id='5Ho1PIYBhwBiCs16RFUh'),
 Video(index='video_index', id='5Xo1PIYBhwBiCs16RFU0'),
 Video(index='video_index', id='5no1PIYBhwBiCs16RFU_')]

In [2]:
from datetime import datetime
from elasticsearch import Elasticsearch
es = Elasticsearch([{"host":"localhost","port":9200}])

doc = {
    'author': 'kimchy',
    'text': 'Elasticsearch: cool. bonsai cool.',
    'timestamp': datetime.now(),
}
res = es.index(index="test-index", id=1, document=doc)
print(res['result'])

res = es.get(index="test-index", id=1)
print(res['_source'])

es.indices.refresh(index="test-index")

res = es.search(index="test-index", query={"match_all": {}})
print("Got %d Hits:" % res['hits']['total']['value'])
for hit in res['hits']['hits']:
    print("%(timestamp)s %(author)s: %(text)s" % hit["_source"])

created
{'author': 'kimchy', 'text': 'Elasticsearch: cool. bonsai cool.', 'timestamp': '2023-02-10T17:17:50.117750'}
Got 1 Hits:
2023-02-10T17:17:50.117750 kimchy: Elasticsearch: cool. bonsai cool.
